#Global Climate Trends
##Analysis

In [27]:
import pandas as pd
import os
import csv

In [52]:
# Path to folders
input_folder_path = os.path.join(".", "input_data")
output_folder_path = os.path.join(".", "output_data")

# Path to input files
pec_input_file_path = os.path.join(input_folder_path, "primary_energy_consumption.csv")

# Path to output files
pec_output_file_path = os.path.join(output_folder_path, "primary_energy_consumption_by_year_n_country.csv")

In [33]:
# Read PEC data and store it in a dataframe
pec_df = pd.read_csv(pec_input_file_path)
pec_df

,Country,1985,1986,1987,1988,1989,1990,1991,1992,1993,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Canada,10.29,10.39,10.72,11.15,11.26,10.83,10.87,11.17,11.45,...,14.40,14.43,14.52,14.32,14.54,14.67,14.59,13.71,13.84,14.14
1,Mexico,4.08,3.96,4.09,4.12,4.36,4.75,4.89,4.98,4.99,...,8.07,8.02,7.93,8.11,8.26,8.16,8.06,7.43,7.99,8.73
2,US,72.66,73.34,75.8,79.25,81.17,81.38,81.27,82.47,84.20,...,92.62,93.59,92.69,92.60,92.95,96.35,95.67,88.57,93.40,95.91
3,Argentina,1.73,1.83,1.91,1.95,1.88,1.85,1.88,2.01,2.08,...,3.53,3.53,3.59,3.57,3.57,3.54,3.34,3.13,3.48,3.60
4,Brazil,4.94,5.21,5.35,5.52,5.67,5.64,5.82,5.90,6.17,...,12.51,12.83,12.66,12.36,12.47,12.51,12.72,12.22,12.85,13.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,Sri Lanka,0.09,0.1,0.1,0.1,0.1,0.11,0.11,0.12,0.13,...,0.29,0.31,0.34,0.38,0.37,0.38,0.39,0.38,0.38,0.34
87,Taiwan,1.43,1.56,1.69,1.86,1.98,2.10,2.23,2.36,2.50,...,4.74,4.84,4.84,4.87,4.86,4.93,4.84,4.70,4.98,4.78
88,Thailand,0.69,0.73,0.83,0.94,1.08,1.27,1.41,1.54,1.75,...,4.71,4.86,4.98,5.08,5.17,5.33,5.34,4.97,5.01,5.06
89,Vietnam,0.22,0.24,0.27,0.27,0.26,0.28,0.28,0.31,0.35,...,2.37,2.62,2.99,3.24,3.48,3.91,4.34,4.34,4.34,4.59


In [44]:
df_array = []

# Loop over the columns of the dataframe skipping the first column
for column in pec_df.columns[1:]:
    # Create a new dataframe with the country and the year
    cy_list = pec_df[['Country', column]].copy()

    # Adding a new column to the dataframe and positionong
    cy_list.insert(0, 'Year', column)

    #Rename for dataset
    cy_list.rename(columns={column: 'Primary Energy Consumption'}, inplace=True)

    # Adding the new dataframe to the array
    df_array.append(cy_list)

df_array
        

[    Year             Country Primary Energy Consumption
 0   1985              Canada                      10.29
 1   1985              Mexico                       4.08
 2   1985                  US                      72.66
 3   1985           Argentina                       1.73
 4   1985              Brazil                       4.94
 ..   ...                 ...                        ...
 86  1985           Sri Lanka                       0.09
 87  1985              Taiwan                       1.43
 88  1985            Thailand                       0.69
 89  1985             Vietnam                       0.22
 90  1985  Other Asia Pacific                       2.07
 
 [91 rows x 3 columns],
     Year             Country Primary Energy Consumption
 0   1986              Canada                      10.39
 1   1986              Mexico                       3.96
 2   1986                  US                      73.34
 3   1986           Argentina                       1.83
 4   

In [55]:
# Merge dfs 
merged_df = pd.concat(df_array, ignore_index=True)
merged_df

#Save csv
merged_df.to_csv(pec_output_file_path, index=False, header=True)

Energy Consumption Per Capita. DF